In [1]:
import os
import pandas as pd
import duckdb
from pathlib import Path

# username = os.getlogin()  # Works locally
# desc_path = os.path.join("C:\\Users", username, "Documents", "GitHub", 
#                          "DEEP_ML_Project", "data", "extracted" , "D_ICD_DIAGNOSES.csv")
# diag_path = os.path.join("C:\\Users", username, "Documents", "GitHub", 
#                          "DEEP_ML_Project", "data", "extracted" , "DIAGNOSES_ICD.csv")
# notes_path = os.path.join("C:\\Users", username, "Documents", "GitHub", 
#                          "DEEP_ML_Project", "data", "extracted" , "NOTEEVENTS.csv")

extracted_dir = Path("../data/extracted")
desc_path = extracted_dir / "D_ICD_DIAGNOSES.csv"
diag_path = extracted_dir / "DIAGNOSES_ICD.csv"
notes_path = extracted_dir / "NOTEEVENTS.csv"

print(desc_path)
print(diag_path)
print(notes_path)

..\data\extracted\D_ICD_DIAGNOSES.csv
..\data\extracted\DIAGNOSES_ICD.csv
..\data\extracted\NOTEEVENTS.csv


In [2]:
# Create an in-memory DuckDB connection
con = duckdb.connect(database=':memory:')

# Load CSVs directly using DuckDB (efficient for large files)
query = f"""
-- Load NOTEEVENTS (discharge summaries only, no known errors)
CREATE VIEW noteevents AS
SELECT *
FROM read_csv_auto('{notes_path}')
WHERE category = 'Discharge summary' AND ISERROR IS NULL;

-- Load DIAGNOSES_ICD
CREATE VIEW diagnoses_icd AS
SELECT *
FROM read_csv_auto('{diag_path}');

-- Load ICD-9 descriptions
CREATE VIEW d_icd_diagnoses AS
SELECT *
FROM read_csv_auto('{desc_path}');
"""

# Run the multi-query
con.execute(query)

In [3]:
# Display NOTEEVENTS column names and top 5 rows
#print("NOTEEVENTS Table Schema:")
#noteevents_schema = con.execute("PRAGMA table_info(noteevents);").fetchdf()
#print(noteevents_schema[['name']])  # Display only column names

#print("\nTop 5 rows from NOTEEVENTS:")
#noteevents_preview = con.execute("SELECT * FROM noteevents LIMIT 5;").fetchdf()
#print(noteevents_preview)

In [4]:
# Get top 20 most frequent ICD-9 codes
top_codes_query = f"""
SELECT icd9_code, COUNT(*) as count
FROM read_csv_auto('{diag_path}')
GROUP BY icd9_code
ORDER BY count DESC
LIMIT 20;
"""
top_codes_df = con.execute(top_codes_query).fetchdf()
top_codes = top_codes_df['ICD9_CODE'].tolist()
#print(top_codes)

# Load ICD-9 descriptions into a DataFrame
desc_df = con.execute("SELECT * FROM d_icd_diagnoses").fetchdf()

# Optional: check column names
print("Top codes columns:", top_codes_df.columns.tolist())
print("Descriptions columns:", desc_df.columns.tolist())

# Try both UPPER and lowercase just in case
count_col = 'COUNT' if 'COUNT' in top_codes_df.columns else 'count'

# Merge with description
merged_df = pd.merge(top_codes_df, desc_df[['ICD9_CODE', 'LONG_TITLE']], on='ICD9_CODE', how='left')

# Sort and print
merged_df = merged_df.sort_values(by=count_col, ascending=False).reset_index(drop=True)

# for idx, row in merged_df.iterrows():
#     print(f"{idx+1}. Code: {row['ICD9_CODE']} → {row['LONG_TITLE']} → {row[count_col]} instances")

Top codes columns: ['ICD9_CODE', 'count']
Descriptions columns: ['ROW_ID', 'ICD9_CODE', 'SHORT_TITLE', 'LONG_TITLE']


In [5]:
# Create a base query to get patient and note data
base_query = f"""
SELECT DISTINCT
    n.subject_id,
    n.hadm_id,
FROM read_csv_auto('{notes_path}') n
JOIN read_csv_auto('{diag_path}') d
  ON n.subject_id = d.subject_id AND n.hadm_id = d.hadm_id
WHERE n.category = 'Discharge summary'
  AND d.icd9_code IN ('{("', '").join(top_codes)}')
"""

# Execute base query to get patient and note data
base_results = con.execute(base_query).fetchdf()
print(f"Base dataset contains {len(base_results)} records")

# Create a dictionary to store diagnosis data for each patient
diagnosis_data = {}

# For each of the top ICD-9 codes, get all patients that have this code
for code in top_codes:
    code_query = f"""
    SELECT DISTINCT
        n.subject_id,
        n.hadm_id
    FROM read_csv_auto('{notes_path}') n
    JOIN read_csv_auto('{diag_path}') d
      ON n.subject_id = d.subject_id AND n.hadm_id = d.hadm_id
    WHERE n.category = 'Discharge summary'
      AND d.icd9_code = '{code}'
    """
    
    # Get all patients with this code
    code_results = con.execute(code_query).fetchdf()
    
    # Create a unique key for each patient admission
    code_results['key'] = code_results['SUBJECT_ID'].astype(str) + '_' + code_results['HADM_ID'].astype(str)
    patient_keys = set(code_results['key'])
    
    # Store this data
    diagnosis_data[code] = patient_keys
    
    print(f"Code {code}: {len(patient_keys)} patients")

# Create one-hot encoding columns
for code in top_codes:
    base_results[f'ICD9_{code}'] = 0
    
# Create a unique key for each patient admission in the base dataset
base_results['key'] = base_results['SUBJECT_ID'].astype(str) + '_' + base_results['HADM_ID'].astype(str)

# Populate the one-hot encoding columns
for code, patient_keys in diagnosis_data.items():
    code_column = f'ICD9_{code}'
    
    # Set 1 for patients that have this diagnosis
    base_results.loc[base_results['key'].isin(patient_keys), code_column] = 1

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Base dataset contains 52586 records


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Code 4019: 20046 patients


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

...

Code 2762: 4358 patients


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Code 496: 4296 patients


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Code 99592: 3792 patients


In [12]:
# Join, filter for discharge notes, keep only top codes
fetch_query = f"""
WITH joined AS (
    SELECT 
        n.subject_id,
        n.hadm_id,
        n.text AS summary_snippet,
        d.icd9_code,
        icd.long_title
    FROM read_csv_auto('{notes_path}') n
    JOIN read_csv_auto('{diag_path}') d
      ON n.subject_id = d.subject_id AND n.hadm_id = d.hadm_id
    JOIN read_csv_auto('{desc_path}') icd
      ON d.icd9_code = icd.icd9_code
    WHERE n.category = 'Discharge summary'
      AND d.icd9_code IN ('{("', '").join(top_codes)}')
)
SELECT 
    subject_id,
    hadm_id,
    MAX(summary_snippet) AS summary_snippet,  -- Use MAX to get a consistent snippet
    STRING_AGG(icd9_code, ', ') AS icd9_codes,
    STRING_AGG(long_title, '; ') AS diagnoses,
    COUNT(icd9_code) AS code_count
FROM joined
GROUP BY subject_id, hadm_id
"""
diagnoses_df = con.execute(fetch_query).fetchdf()

# Add diagnosis text fields and full summary text
diagnoses_df['key'] = diagnoses_df['SUBJECT_ID'].astype(str) + '_' + diagnoses_df['HADM_ID'].astype(str)

# Join the full data to the base results
base_results['key'] = base_results['SUBJECT_ID'].astype(str) + '_' + base_results['HADM_ID'].astype(str)
base_results = base_results.join(
    diagnoses_df.set_index('key')[['summary_snippet', 'icd9_codes', 'diagnoses', 'code_count']], 
    on='key'
)

# Join the full data to the base results
base_results['key'] = base_results['SUBJECT_ID'].astype(str) + '_' + base_results['HADM_ID'].astype(str)
base_results = base_results.join(
    diagnoses_df.set_index('key')[['summary_snippet', 'icd9_codes', 'diagnoses', 'code_count']], 
    on='key'
)


# Remove the temporary key column
results = base_results.drop(columns=['key'])


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [13]:
# #Print the results of the summary and which codes were assocated with it Example
# print(results.loc[5, 'summary_snippet'])
# print("-" * 20)  # Separator for clarity

# icd9_codes = results.loc[1, 'icd9_codes'].split(', ')
# diagnoses = results.loc[1, 'diagnoses'].split('; ')

# for i in range(min(len(icd9_codes), len(diagnoses))):
#   print(f"{icd9_codes[i]} --> {diagnoses[i]}")

# # handling the case where icd9_codes and diagnoses are different lengths.
# if len(icd9_codes) > len(diagnoses):
#     for i in range(len(diagnoses), len(icd9_codes)):
#         print(f"{icd9_codes[i]} --> No corresponding diagnosis")
# elif len(diagnoses) > len(icd9_codes):
#     for i in range(len(icd9_codes), len(diagnoses)):
#         print(f"No corresponding ICD9 --> {diagnoses[i]}")


In [11]:
# Export results to CSV
#summary_results_path = os.path.join("C:\\Users", username, "Documents", "GitHub", "DEEP_ML_Project", "data", "preprocessed", "summary_results.csv")
#summary_results_trimmed_path = os.path.join("C:\\Users", username, "Documents", "GitHub", "DEEP_ML_Project", "data", "preprocessed", "summary_results_trimmed.csv")

preprocessed_dir = Path("../data/preprocessed")
preprocessed_dir.mkdir(parents=True, exist_ok=True)

summary_results_path = preprocessed_dir / "summary_results.csv"
summary_results_trimmed_path = preprocessed_dir / "summary_results_trimmed.csv"

results.iloc[:-20].to_csv(summary_results_path, index=False)
results.iloc[-20:].to_csv(summary_results_trimmed_path, index=False)

print(f"Full training results exported to: {summary_results_path}")
print(f"Trimmed training results exported to: {summary_results_trimmed_path}")

Full training results exported to: ..\data\preprocessed\summary_results.csv
Trimmed training results exported to: ..\data\preprocessed\summary_results_trimmed.csv


In [ ]:
# # Sample JOIN query to fetch ICD-9 + Discharge Summary
# fetch_query = f"""
# SELECT 
#     n.subject_id,
#     n.hadm_id,
#     SUBSTRING(n.text, 1, 4000) AS summary_snippet,
#     STRING_AGG(d.icd9_code, ', ') AS icd9_codes,
#     STRING_AGG(icd.long_title, '; ') AS diagnoses
# FROM read_csv_auto('{notes_path}') n
# JOIN read_csv_auto('{diag_path}') d
#   ON n.subject_id = d.subject_id AND n.hadm_id = d.hadm_id
# JOIN read_csv_auto('{desc_path}') icd
#   ON d.icd9_code = icd.icd9_code
# WHERE n.category = 'Discharge summary'
# GROUP BY n.subject_id, n.hadm_id, n.text
# # LIMIT 5000;
# """



# # Execute and display results
# results = con.execute(fetch_query).fetchdf()
# print(results.head(3))  # Show sample output